In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
r1 = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r1.content,'html.parser')
table = soup.find(lambda tag: tag.name =='table' and ("wikitable" in tag['class']))

In [3]:
df = pd.read_html(str(table), flavor='bs4')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleanup table data

In [5]:
df_2 = (df['Borough'] == 'Not assigned')|(df['Neighbourhood'] == 'Not assigned')
df = df[~df_2]

In [6]:
(df.Borough == 'Not assigned').sum()

0

In [7]:
(df.Neighbourhood == 'Not Assigned').sum()

0

In [8]:
post=df.Postcode.unique()

## Group by postcode

In [9]:
Toronto = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
for code in post:
    temp_df = df[['Borough','Neighbourhood']][df['Postcode'] == code]
    boro = temp_df.Borough.unique()
    hood = temp_df.Neighbourhood.unique()
    Toronto = Toronto.append({
        'Postcode':code,
        'Borough':",".join(boro),
        'Neighbourhood':",".join(hood)},ignore_index=True)

In [10]:
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [11]:
Toronto.shape

(103, 3)

## Include lat/long data from geocode

In [13]:
postalcodes_from_csv = pd.read_csv('http://cocl.us/Geospatial_data')
postalcodes_from_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
Toronto = Toronto.sort_values(by=['Postcode'])
Toronto.reset_index(inplace=True, drop=True)
pcode = postalcodes_from_csv.sort_values(by=['Postal Code'])
pcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
Toronto=pd.concat([Toronto, pcode[['Latitude','Longitude']]], axis = 1)
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Filter only Boroughs in Toronto

In [17]:
Toronto = Toronto[Toronto.Borough.str.contains('Toronto', na=False)]
Toronto.reset_index( inplace = True)
Toronto

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [18]:
Toronto.drop(['index'], axis = 1, inplace = True)
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [20]:
!pip install folium

     |████████████████████████████████| 92kB 9.2MB/s eta 0:00:011


## Create folium map

In [21]:
import folium

map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

## Extract information from FourSquare

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json

def getNearbyVenues(postcodes, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    postcodes_done = ""
    for code, lat, lng in zip(postcodes, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        res = requests.get(url).json()["response"]
        if 'groups' not in res:
            continue;
        
        postcodes_done += code+","
        results = res['groups'][0]['items']
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print(postcodes_done[:-1])
    return(nearby_venues)

## Include venues into dataframe

In [24]:
toronto_venues = getNearbyVenues(Toronto['Postcode'], Toronto['Latitude'], Toronto['Longitude'],radius=500, limit=100)

M4E,M4K,M4L,M4M,M4N,M4P,M4R,M4S,M4T,M4V,M4W,M4X,M4Y,M5A,M5B,M5C,M5E,M5G,M5H,M5J,M5K,M5L,M5N,M5P,M5R,M5S,M5T,M5V,M5W,M5X,M6G,M6H,M6J,M6K,M6P,M6R,M6S,M7A,M7Y


In [25]:
toronto_venues.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## One-hot encoding

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1715, 234)

In [28]:
toronto_grouped = toronto_onehot.groupby('Postcode').sum()
toronto_grouped.reset_index(inplace=True)
toronto_grouped.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4K,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4M,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,1
4,M4N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Find Restaurant of choice

In [31]:
df2 = toronto_grouped[['Postcode','Afghan Restaurant']]
df2

,Postcode,Afghan Restaurant
0,M4E,0
1,M4K,0
2,M4L,0
3,M4M,0
4,M4N,0
5,M4P,0
6,M4R,0
7,M4S,0
8,M4T,0
9,M4V,0


## Load csv file

In [36]:
# The code was removed by Watson Studio for sharing.

,Topic,Characteristic,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park,Unnamed: 142
0,Income of economic families in 2015,Total - Economic family income decile group fo...,"28,825","23,475","12,030","28,650","27,005","15,580","25,600","21,135",...,"22,150","53,005","12,430","7,850","13,200","11,810","12,370","27,570","14,020",NaN
1,Income of economic families in 2015,In the bottom half of the distribution,"18,535","15,205","4,735","12,080","10,760","8,150","15,230","10,745",...,"10,895","36,245","5,025","3,490","6,355","3,840","4,165","19,105","7,735",NaN
2,Income of economic families in 2015,In the bottom decile,"4,195","3,905",725,"4,810","2,600","1,825","8,820","4,005",...,"2,005","9,965","1,490",890,"1,685","1,185","1,340","5,410","1,630",NaN
3,Income of economic families in 2015,In the second decile,"5,030","3,565",815,"2,225","2,195","1,865","2,020","1,995",...,"2,450","8,715","1,040",715,"1,480",660,675,"4,135","1,665",NaN
4,Income of economic families in 2015,In the third decile,"3,700","2,940","1,005","1,710","2,050","1,595","1,660","1,585",...,"2,295","6,815",850,625,"1,175",630,715,"3,625","1,775",NaN


In [45]:
neighborhoods_id = df_data_0
neighborhoods_id.head()

,Topic,Characteristic,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park,Unnamed: 142
2,Income of economic families in 2015,In the bottom decile,"4,195","3,905",725,"4,810","2,600","1,825","8,820","4,005",...,"2,005","9,965","1,490",890,"1,685","1,185","1,340","5,410","1,630",NaN
3,Income of economic families in 2015,In the second decile,"5,030","3,565",815,"2,225","2,195","1,865","2,020","1,995",...,"2,450","8,715","1,040",715,"1,480",660,675,"4,135","1,665",NaN
4,Income of economic families in 2015,In the third decile,"3,700","2,940","1,005","1,710","2,050","1,595","1,660","1,585",...,"2,295","6,815",850,625,"1,175",630,715,"3,625","1,775",NaN
5,Income of economic families in 2015,In the fourth decile,"2,990","2,590","1,060","1,630","1,875","1,455","1,380","1,550",...,"2,120","5,845",780,610,965,665,715,"3,235","1,380",NaN
6,Income of economic families in 2015,In the fifth decile,"2,610","2,210","1,140","1,735","2,045","1,410","1,350","1,620",...,"2,030","4,955",870,645,"1,060",700,730,"2,710","1,290",NaN


In [46]:
neighborhoods_id = neighborhoods_id.transpose()
neighborhoods_id.head()

,2,3,4,5,6,7,8,9,10,11,12,13
Topic,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Ethnic origin population
Characteristic,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Agincourt North,"4,195","5,030","3,700","2,990","2,610","10,280","2,500","2,340","2,260","1,900","1,305",85
Agincourt South-Malvern West,"3,905","3,565","2,940","2,590","2,210","8,265","2,015","1,840","1,800","1,505","1,125",255
Alderwood,725,815,"1,005","1,060","1,140","7,290","1,265","1,420","1,500","1,750","1,345",10


In [47]:
neighborhoods_id.drop(neighborhoods_id.index[0], inplace = True)
neighborhoods_id.head()

,2,3,4,5,6,7,8,9,10,11,12,13
Characteristic,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Agincourt North,"4,195","5,030","3,700","2,990","2,610","10,280","2,500","2,340","2,260","1,900","1,305",85
Agincourt South-Malvern West,"3,905","3,565","2,940","2,590","2,210","8,265","2,015","1,840","1,800","1,505","1,125",255
Alderwood,725,815,"1,005","1,060","1,140","7,290","1,265","1,420","1,500","1,750","1,345",10
Annex,"4,810","2,225","1,710","1,630","1,735","16,570","1,720","1,820","2,210","2,830","7,995",35


In [48]:
neighborhoods_id.sort_values(by=[13])

,2,3,4,5,6,7,8,9,10,11,12,13
Lawrence Park South,930,500,460,435,565,"12,200",650,685,"1,005","1,475","8,385",0
North Riverdale,"1,000",880,570,710,770,"7,745",755,820,"1,050","1,555","3,555",0
Palmerston-Little Italy,"2,055","1,340","1,150","1,060",935,"7,205","1,145","1,090","1,330","1,395","2,250",0
Playter Estates-Danforth,790,635,480,410,505,"4,825",535,620,620,865,"2,185",0
Roncesvalles,"2,405","1,635","1,220","1,085","1,090","7,440","1,100","1,165","1,280","1,575","2,325",0
Corso Italia-Davenport,"1,620","1,370","1,350","1,390","1,390","6,965","1,410","1,405","1,510","1,440","1,185",0
Markland Wood,575,610,840,720,750,"6,960",925,"1,070","1,250","1,575","2,145",0
Rosedale-Moore Park,"1,750",950,740,750,870,"15,555","1,025","1,170","1,460","2,120","9,760",0
Runnymede-Bloor West Village,515,495,485,560,595,"7,410",725,945,990,"1,735","3,010",0
Casa Loma,"1,085",690,545,515,560,"7,395",530,725,755,"1,080","4,305",0
